In [1]:
import pandas as pd
import requests as r

In [2]:
anime_data = []

In [3]:
for pages in [1,2,3,4,5,6,7,8]:
    url = 'https://api.jikan.moe/v4/top/anime?page=' + str(pages)
    mal_anime = r.get(url)
    jmal_anime = mal_anime.json()
    anime_data = anime_data+jmal_anime['data']

In [4]:
dfa = pd.json_normalize(anime_data)
dfa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 57 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   mal_id                            200 non-null    int64  
 1   url                               200 non-null    object 
 2   title                             200 non-null    object 
 3   title_english                     167 non-null    object 
 4   title_japanese                    200 non-null    object 
 5   title_synonyms                    200 non-null    object 
 6   type                              200 non-null    object 
 7   source                            200 non-null    object 
 8   episodes                          196 non-null    float64
 9   status                            200 non-null    object 
 10  airing                            200 non-null    bool   
 11  duration                          200 non-null    object 
 12  rating  

In [5]:
clear()

In [6]:
cols=[0,1,3,4,5,6,7,10,11,14,16,17,18,19,20,23,24,27,28,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56]
dfa_new = dfa.drop(dfa.columns[cols],axis=1,inplace=False)

In [7]:
dfa_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         200 non-null    object 
 1   episodes      196 non-null    float64
 2   status        200 non-null    object 
 3   rating        200 non-null    object 
 4   score         200 non-null    float64
 5   rank          200 non-null    int64  
 6   season        132 non-null    object 
 7   year          132 non-null    float64
 8   studios       200 non-null    object 
 9   genres        200 non-null    object 
 10  demographics  200 non-null    object 
dtypes: float64(3), int64(1), object(7)
memory usage: 17.3+ KB


In [8]:
dfa_new["season"].fillna(value = 'Out of Season', inplace = True)

In [9]:
missing_mask = dfa_new['episodes'].isna()
# Finds all NaN values in 'episodes' column and determines whether there are NaN values (boolean T/F)
mapping_dict = dict({'Currently Airing':'Currently Airing'})
# creates a dicitionary which finds all 'Currently Aiting' values and will allow for those values to be replaced to 'Currently Airing'
dfa_new.loc[missing_mask,'episodes'] = dfa_new.loc[missing_mask,'status'].map(mapping_dict)
# creates a slice of the 'episodes' column which have NaN values and creates a loop with the map command
# to go through the entire column
dfa_new['episodes'] = dfa_new['episodes'].fillna('Currently Airing')
# uses the fillna command to fill all NaN values with 'Currently Airing'

In [10]:
dfa_new['studios'][0]

[{'mal_id': 4,
  'type': 'anime',
  'name': 'Bones',
  'url': 'https://myanimelist.net/anime/producer/4/Bones'}]

In [11]:
dfa_new['studios'] = dfa_new['studios'].map(lambda studios: [studio['name'] for studio in studios])
# takes the first value of the 'studios' column and creates a map 
# similar to the above one where it loops through the entire column.
# the map allows for a function which uses a custom function (lambda) to take only the studios' name in the dictionary

In [12]:
dfa_new['genres'] = dfa_new['genres'].map(lambda genres: [genre['name'] for genre in genres])

In [13]:
dfa_new['demographics'] = dfa_new['demographics'].map(lambda demographics: [demographic['name'] for demographic in demographics])

In [14]:
dfa_new.to_excel(r'C:\Users\hirok\OneDrive\Documents\Data Analysis\MAL Project\dfa_new.xlsx',index = False, header = True)

In [15]:
dfa_new

,title,episodes,status,rating,score,rank,season,year,studios,genres,demographics
0,Fullmetal Alchemist: Brotherhood,64.0,Finished Airing,R - 17+ (violence & profanity),9.14,1,spring,2009.0,[Bones],"[Action, Adventure, Drama, Fantasy]",[Shounen]
1,Gintama°,51.0,Finished Airing,PG-13 - Teens 13 or older,9.09,2,spring,2015.0,[Bandai Namco Pictures],"[Action, Comedy, Sci-Fi]",[Shounen]
2,Spy x Family,12.0,Currently Airing,PG-13 - Teens 13 or older,9.09,3,spring,2022.0,"[Wit Studio, CloverWorks]","[Action, Comedy]",[Shounen]
3,Shingeki no Kyojin Season 3 Part 2,10.0,Finished Airing,R - 17+ (violence & profanity),9.08,4,spring,2019.0,[Wit Studio],"[Action, Drama]",[Shounen]
4,Steins;Gate,24.0,Finished Airing,PG-13 - Teens 13 or older,9.08,5,spring,2011.0,[White Fox],"[Drama, Sci-Fi, Suspense]",[]
...,...,...,...,...,...,...,...,...,...,...,...
195,Kiseijuu: Sei no Kakuritsu,24.0,Finished Airing,R - 17+ (violence & profanity),8.35,196,fall,2014.0,[Madhouse],"[Action, Horror, Sci-Fi]",[Seinen]
196,Psycho-Pass,22.0,Finished Airing,R - 17+ (violence & profanity),8.35,197,fall,2012.0,[Production I.G],"[Action, Sci-Fi, Suspense]",[]
197,Natsume Yuujinchou: Itsuka Yuki no Hi ni,1.0,Finished Airing,PG-13 - Teens 13 or older,8.34,198,Out of Season,NaN,[Brain's Base],"[Drama, Slice of Life, Supernatural]",[Shoujo]
198,Neon Genesis Evangelion,26.0,Finished Airing,PG-13 - Teens 13 or older,8.34,200,fall,1995.0,"[Gainax, Tatsunoko Production]","[Action, Avant Garde, Drama, Sci-Fi]",[]


In [16]:
dfa_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         200 non-null    object 
 1   episodes      200 non-null    object 
 2   status        200 non-null    object 
 3   rating        200 non-null    object 
 4   score         200 non-null    float64
 5   rank          200 non-null    int64  
 6   season        200 non-null    object 
 7   year          132 non-null    float64
 8   studios       200 non-null    object 
 9   genres        200 non-null    object 
 10  demographics  200 non-null    object 
dtypes: float64(2), int64(1), object(8)
memory usage: 17.3+ KB


In [17]:
colz = [1,2,3,4,6,7,8]

In [18]:
dfa_use = dfa_new.drop(dfa_new.columns[colz],axis=1,inplace=False)

In [19]:
dfa_use.head()

,title,rank,genres,demographics
0,Fullmetal Alchemist: Brotherhood,1,"[Action, Adventure, Drama, Fantasy]",[Shounen]
1,Gintama°,2,"[Action, Comedy, Sci-Fi]",[Shounen]
2,Spy x Family,3,"[Action, Comedy]",[Shounen]
3,Shingeki no Kyojin Season 3 Part 2,4,"[Action, Drama]",[Shounen]
4,Steins;Gate,5,"[Drama, Sci-Fi, Suspense]",[]


In [20]:
dfa_use.to_excel(r'C:\Users\hirok\OneDrive\Documents\Data Analysis\MAL Project\dfa_use.xlsx',index = False, header = True)